In [ ]:
%load_ext autoreload
%autoreload 2
%run ./initializer

In [ ]:
import pandas as pd
from lgbm_ranker import LGBMRankerModel, LGBMRanker
from feature_tester import FeatureTester
import numpy as np

In [ ]:
train_df = pd.read_parquet("data/training_set_processed.parquet")

In [ ]:
lgbm_class_model = LGBMRankerModel(
    df = train_df,              # Only initialized to get column names
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[5],
    n_estimators=1000,
    learning_rate=0.05,
)

In [ ]:
impute_null_columns = {"visitor_hist_starrating": "median", "visitor_hist_adr_usd": "mean", "prop_review_score": "median", "orig_destination_distance": "mean", "prop_log_historical_price":"mean"}
feature_tester = FeatureTester(lgbm_class_model, impute_null_columns=impute_null_columns)

In [ ]:
model_opt, model_opt_not_imputed = feature_tester.test_na_fill_method()